<a href="https://colab.research.google.com/github/julieisok/OurFisrtML/blob/master/%EB%8D%B0%EC%9D%B4%ED%84%B0%ED%9A%8D%EB%93%9D_level3_webscraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 웹스크래핑을 이용한 데이터 수집

* open API를 이용한 데이터 수집이 불가한 경우 해당
* 웹을 요청하여 html을 모두 받아서 DOM으로 올려서 데이터를 추출방식
* DOM을 띠워서 데이터를 추출할때 사용하는 라이브러리
 > beautifulsoup (bs4)를 활용  
 > conda install beautifulsoup4

In [0]:
from bs4 import BeautifulSoup

In [0]:
from urllib.request import urlopen

- 콘텐츠가 존재하는 해당 페이지까지 진입
- 진입간에 로그인, ajax등등 사람의 손을 타지 않는지 체크
- 그냥 url만 넣으면 화면이 구성된다 => OK
- html 자체에 프레임이 적용된경우 실제 주소까지 찾아서 이동
- 통신시 get, post등 데이터를 전달해서 획득하는 것도 OK

### 네이버 금융 > 고시환율정보 수집

- 하루에 환율이 수시로 변경된다 
- 그 주기를 관찰하여 수집 주기를 결정해야 한다

In [0]:
target_site = 'https://finance.naver.com/marketindex/exchangeList.nhn'
target_site

'https://finance.naver.com/marketindex/exchangeList.nhn'

In [0]:
# 요청및 응답 획득
page = urlopen( target_site )
page

In [0]:
# DOM  구성
# 'html5lib' 파서는 html  양이 크거나, 정교한 파싱을 할때 즉, 아래 파서가
# 정상적으로 결과를 내지 못하면 이 파서로 교체
soup = BeautifulSoup( page,  'html.parser')

In [0]:
# 리스트 내포를 통해서 간단하게 가공해 보았다
tmp = [ td.a.string.strip() for td in soup.find_all('td','tit') ]
tmp[:3]

['미국 USD', '유럽연합 EUR', '일본 JPY (100엔)']

In [0]:
for td in soup.find_all('td','tit'):
    print( td.a.string.strip() )
    break

미국 USD


- 데이터를 추출하여 디비에 적제하기 위해 최종 형태는 다음과 같다
- [
    {
      'name':'미국 USD',
      'code';'USD',
      'buy_std_rate':1209.90,
      'cash_sell':1188.73
    },{}
 ]

In [0]:
'1,234'.replace(',','')

'1234'

In [0]:
# 통화
tmp = [ td.a.string.strip() for td in soup.find_all('td','tit') ]
tmp[:3]

['미국 USD', '유럽연합 EUR', '일본 JPY (100엔)']

In [0]:
# 매매기준율
tmp = [ td.string.strip() for td in soup.find_all('td','sale') ]
tmp[:3]

['1,199.00', '1,309.67', '1,111.06']

In [0]:
body > div > table > tbody > tr:nth-child(1) > td:nth-child(4)

- 만약 대상 페이지에 table이 n개 존재하면 특정해서 찾아야한다
- 대상 데이터가 있는 tr을 모두 찾았다
- table.tbl_exchange > tbody > tr
- 위의 표현을 for문으로 구동하고 하나하나의 tr에서 td:nth-child(4) 를 선택

In [0]:
# 원하는 데이터를 감사는 tr를 찾아서 그밑에서 자식들을 탐색후 거기서 데이터를 추출!!
for tr in soup.select('table.tbl_exchange > tbody > tr'):
    # 데이터 한줄 한줄 뽑아서 => tr에서 탐색하여 세부 데이터 추출
    print( tr.select_one('td.tit').a.string.strip() )
    print( tr.select_one('td.sale').string.strip() )
    print( tr.select_one('td:nth-child(4)').string.strip() )

미국 USD
1,209.40
1,188.24
유럽연합 EUR
1,353.02
1,326.10
일본 JPY (100엔)
1,141.64
1,121.67
중국 CNY
170.74
162.21
홍콩 HKD
154.22
151.19
대만 TWD
38.69
35.99
영국 GBP
1,468.70
1,439.77
오만 OMR
3,141.22
2,952.75
캐나다 CAD
914.55
896.54
스위스 CHF
1,241.62
1,217.17
스웨덴 SEK
126.05
122.97
호주 AUD
824.15
807.92
뉴질랜드 NZD
784.36
768.91
체코 CZK
52.47
47.75
칠레 CLP
1.70
1.57
터키 TRY
220.50
0.00
몽골 MNT
0.45
0.00
이스라엘 ILS
347.60
319.80
덴마크 DKK
181.33
176.89
노르웨이 NOK
135.49
132.18
사우디아라비아 SAR
322.37
300.13
쿠웨이트 KWD
3,976.33
3,658.23
바레인 BHD
3,207.96
2,951.33
아랍에미리트 AED
329.25
306.54
요르단 JOD
1,705.78
1,569.32
이집트 EGP
73.01
0.00
태국 THB
39.35
36.99
싱가포르 SGD
875.01
857.60
말레이시아 MYR
288.98
267.60
인도네시아 IDR 100
8.53
7.68
카타르 QAR
330.45
0.00
카자흐스탄 KZT
3.12
0.00
브루나이 BND
875.05
822.55
인도 INR
17.16
0.00
파키스탄 PKR
7.56
0.00
방글라데시 BDT
14.33
0.00
필리핀 PHP
23.29
21.39
멕시코 MXN
62.25
57.03
브라질 BRL
308.59
283.91
베트남 VND 100
5.21
4.60
남아프리카 공화국 ZAR
80.37
73.95
러시아 RUB
18.57
16.53
헝가리 HUF
4.17
3.84
폴란드 PLN
312.96
287.93


In [0]:
results = []
for tr in soup.select('table.tbl_exchange > tbody > tr'):
    if tr.select_one('td.tit').a.string.strip().count('JPY') == 0:
        tmp   = dict()
        tmp['name']         = tr.select_one('td.tit').a.string.strip()
        tmp['code']         = tr.select_one('td.tit').a['href'][-6:-3]
        tmp['buy_std_rate'] = tr.select_one('td.sale').string.strip() 
        tmp['cash_sell']    = tr.select_one('td:nth-child(4)').string.strip()
        # :nth-child(4)가 않되는 분은 아래로 처리변경
        #tmp['cash_sell']    = tr.select('td')[3].string.strip()
        results.append( tmp )
results

[{'name': '미국 USD',
  'code': 'USD',
  'buy_std_rate': '1,209.40',
  'cash_sell': '1,188.24'},
 {'name': '유럽연합 EUR',
  'code': 'EUR',
  'buy_std_rate': '1,353.02',
  'cash_sell': '1,326.10'},
 {'name': '일본 JPY (100엔)',
  'code': 'JPY',
  'buy_std_rate': '1,141.64',
  'cash_sell': '1,121.67'},
 {'name': '중국 CNY',
  'code': 'CNY',
  'buy_std_rate': '170.74',
  'cash_sell': '162.21'},
 {'name': '홍콩 HKD',
  'code': 'HKD',
  'buy_std_rate': '154.22',
  'cash_sell': '151.19'},
 {'name': '대만 TWD',
  'code': 'TWD',
  'buy_std_rate': '38.69',
  'cash_sell': '35.99'},
 {'name': '영국 GBP',
  'code': 'GBP',
  'buy_std_rate': '1,468.70',
  'cash_sell': '1,439.77'},
 {'name': '오만 OMR',
  'code': 'OMR',
  'buy_std_rate': '3,141.22',
  'cash_sell': '2,952.75'},
 {'name': '캐나다 CAD',
  'code': 'CAD',
  'buy_std_rate': '914.55',
  'cash_sell': '896.54'},
 {'name': '스위스 CHF',
  'code': 'CHF',
  'buy_std_rate': '1,241.62',
  'cash_sell': '1,217.17'},
 {'name': '스웨덴 SEK',
  'code': 'SEK',
  'buy_std_rate': '

In [0]:
results = [ {
    'name':tr.select_one('td.tit').a.string.strip(),
    'code':tr.select_one('td.tit').a['href'][-6:-3],
    'buy_std_rate':tr.select_one('td.sale').string.strip(),
    'cash_sell':tr.select_one('td:nth-child(4)').string.strip()
} for tr in soup.select('table.tbl_exchange > tbody > tr') 
  if tr.select_one('td.tit').a.string.strip().count('JPY') == 0
]

In [0]:
results[:2]

[{'name': '미국 USD',
  'code': 'USD',
  'buy_std_rate': '1,209.40',
  'cash_sell': '1,188.24'},
 {'name': '유럽연합 EUR',
  'code': 'EUR',
  'buy_std_rate': '1,353.02',
  'cash_sell': '1,326.10'}]